In [ ]:
# Import Necessary Modules for Data Preprocessing

# Used for loading in training data [Function #1 - load_raw_training_data()]
import pandas as pd
# Adjust column width settings to see all of the 'original_text' column
pd.set_option('max_colwidth', 400)

# Used for replacing '-LRB-' and '-RRB-' with left and right parentheses in original text repectively [Function #2 - replace_LRB_and_RRB()]
import re

# Used for label value changing in preprocessing training data [Function #6 - preprocessing_training_data()]
import numpy as np

# Used for tokenization when creating score values against extraneous resourses [Function #8 - extraneous_score_calculation()]
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize 

# Used for Parts-of-Speech tagging [Function #14 - POS_preprocessing()]
from nltk.tag import pos_tag

# Used for Lemmatization [Function #15 - lemma_preprocessing()]
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Used for vectorization [Function # ]
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
# Load Raw WikiLarge Training Data from GitHub Repository
def load_raw_training_data():
  # WikiLarge Training Data is very large and was split into three CSV files load each of them in.
  textData_1 = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/WikiLarge_Train_part_1.csv')
  textData_2 = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/WikiLarge_Train_part_2.csv')
  textData_3 = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/WikiLarge_Train_part_3.csv')

  # Concatenate each of the parts together to get the original data in one dataframe
  text_data = pd.concat([textData_1, textData_2, textData_3], ignore_index=True)

  # Return concatenated dataframe
  return text_data

# **Replacing -LRB- and -RRB-**
Notice the fifth entry in the 'original_text' column of the text_data dataframe. <br>
![](https://drive.google.com/uc?id=1UCPuQrnQfSWAffp_XNTw916c8o4YgicV) <br> <br>
Now Compare that with the wikipedia page regarding Geneva and the text of the first paragraph.
![](https://drive.google.com/uc?id=1rxkFMR21K17lCM0-rOUL2x631se-fgxQ) <br> <br>
Based off of this, we have determined that '-LRB- and '-RRB-' are the left and right parentheses respectively. 

In [ ]:
# After searching for the first few 'original_text' entries presented in text_data.head() - it was determined
# that '-LRB-' and '-RRB-' are left and right parentheses respectively. This function replaces those
# text strings with their respective symbols.
def replace_LRB_and_RRB(text):
  # Replace the substring "-LRB-" with "(" in input string
  new_string = re.sub("-LRB-", "(", text)

  # Replace the substring "-RRB-" with ")" in new_string
  second_string = re.sub("-RRB-", ")", new_string)

  # Return the output of the second replacement
  return second_string

In [ ]:
# Based on the finding of '-LRB-' and '-RRB-', this function replaces every character in a string that is not a parentheses with no text
def find_parentheses(text):
  punctuation_string = obtain_non_Alphanumeric(text)
  new_string = re.sub("[^()]", "", punctuation_string)
  return new_string

In [ ]:
# Function to find any uneven parentheses within the 'original_text' column
def determine_uneven_parentheses(text):
  # Use previously generated function to find all parentheses within the 'original_text' column
  parentheses_only = find_parentheses(text)

  # Generate a list containing a single string of a closed parentheses
  parentheses_string_list = ['()']

  # While any closed parentheses exist in the parentheses column
  while any(x in parentheses_only for x in parentheses_string_list):
    # Replace the closed parentheses with no text
    for paren in parentheses_string_list:
      parentheses_only = parentheses_only.replace(paren, "")

  # Output result as a boolean to determine if string parentheses_only has been reduce to an empty string
  result = not parentheses_only

  # Return boolean value as 0 or 1 - 0 indicating that the 'original_text' column has closed parentheses
  if result == False:
    return 1
  else:
    return 0

In [ ]:
# Function to obtain any non-alphanumeric characters
def obtain_non_Alphanumeric(text):
  # replace all non-alphanumeric characters of an input string with no text to a new output called 'new_string'
  new_string, number_of_subs = re.subn("[a-zA-Z0-9]", "", text)

  # replace all white space characters of 'new_string' with no text to the output 'second_string'
  second_string, second_subs = re.subn("\s", "", new_string)

  # return 'second_string'
  return second_string

# **Duplicate Values**
In the WikiLarge training data - it was found that there were repeated entries of the 'original_text' column which had opposing label values. The following function takes all duplicate values and compared the total number of entries that label an 'original_text' sample as not requiring simplication against the total number of entries that label the sample as requiring simplification. The function then reassigns the duplicate values as a single entry with the more prominent label. If there is an even number of both labels, then the sample is not included in the post processed training data.

In [ ]:
# Combined previous functions into large preprocessing function

# Additionally - find duplicate 'original_text' entries, find the mean of their label values - as some of them have
# opposing label values - then remove all duplicates except for one with the mean label value adjusted to 0 or 1
# based on rounding
def preprocessing_training_data():
  text_data = load_raw_training_data()

  # Convert binary 0 labels to -1 (this helps with keeping some of the duplicate entries by mean value calculation)
  text_data['label'] = np.where(text_data['label'] < 1, -1, 1)

  # Separate out the duplicate entries from the individual entries - All duplicate entries are taken to "duplicate_texts" dataframe
  duplicate_texts = text_data[text_data.original_text.duplicated(keep=False)].copy()

  # Individual entries can be used directly in the final version of the preprocessed dataframe
  individual_texts = text_data[~text_data.original_text.duplicated(keep=False)].copy()
  
  # Group the duplicate text entries by the original text and find the mean value. 
  # If the mean value is negative, then most of the entries have been labeled as -1
  # If the mean value is positive, then most of the entries have been labeled as 1
  dup_group = duplicate_texts.groupby(['original_text'], as_index=False).mean()
  
  # Convert all positive values to 1 and all negative values to -1
  dup_group['label'] = np.where(dup_group['label'] > 0, 1, dup_group['label'])
  dup_group['label'] = np.where(dup_group['label'] < 0, -1, dup_group['label'])
  
  # If the label mean is 0, then it is an even split, and the text data cannot be used for classification
  # Identify all rows with mean groupby label values of 0
  zero_mean = dup_group[dup_group['label'] == 0]
  
  # Identify all rows with positive groupby label values
  pos_mean = dup_group[dup_group['label'] > 0].copy()

  # Perform the same for all rows with negative groupby label values
  neg_mean = dup_group[dup_group['label'] < 0].copy()
  
  # Recombine the acceptable duplicate entries with the original individual entries
  new_text_data = pd.concat([pos_mean, neg_mean, individual_texts], ignore_index=True)

  # Convert the labels of -1 back to 0 as in the original training data
  new_text_data['label'] = np.where(new_text_data['label'] < 0, 0, 1)

  # Replace "-LRB-" and "-RRB-" with left and right parentheses
  new_text_data['original_text'] = new_text_data.original_text.apply(lambda x: replace_LRB_and_RRB(x))

  # Create a Column of only the punctuation using previously made function
  new_text_data['punctuation'] = new_text_data.original_text.apply(lambda x: obtain_non_Alphanumeric(x))

  # Determine if a text entry has closed parentheses or not
  new_text_data['closed_parentheses'] = new_text_data.original_text.apply(lambda x: determine_uneven_parentheses(x))
  
  return new_text_data

In [ ]:
# Function which loads external resouce data provided with the WikiLarge data - those resources are:
# 1) The Dale Chall 3000 Word List, which is one definition of words that are considered "basic" English.
# 2) "Age of Acquisition" (AoA) estimates for about 51k English words, which refers to the approximate age (in years) when a word was learned. Early words, being more basic, have lower average AoA.
# 3) Brysbaert et al Concreteness Ratings for 40 thousand English lemma words gathered via 
#    Amazon Mechanical Turk. The ratings come from a larger list of 63 thousand words and represent all English words known to 85% of the raters.

def load_external_resource_data():
  # Load Dale Chall word list as a list of strings
  dale_chall = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/dale_chall.txt', header=None)
  d_c_df = dale_chall.rename(columns={0:'words'})
  d_c_list = d_c_df['words'].to_list()

  # Load AoA estimates for about 51 thousand English words, and return it as a dictionary
  AoA = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/AoA_51715_words.csv', encoding='unicode_escape')
  # Reduce the dataframe to the word and the AoA_Kup_lem score
  AoA = AoA[['Word', 'AoA_Kup_lem']]
  # Drop any rows where the AoA_Kup_lem score is not a value
  AoA = AoA[AoA['AoA_Kup_lem'].notna()]
  # Set the index of the dataframe to the words
  AoA = AoA.set_index('Word')
  # Take the AoA_Kup_lem score series out as a dictionary
  AoA_dict = AoA['AoA_Kup_lem'].to_dict()

  # Load Brysbaert Concreteness ratings, and return it as a dictionary
  Brysbaert = pd.read_csv('https://raw.githubusercontent.com/nruloff/Difficulty_Classification_of_Textual_Passages/main/Data/Concreteness_ratings_Brysbaert_et_al_BRM.txt', delimiter='\t')
  # Reduce the dataframe to the word and Concreteness rating
  Brysbaert = Brysbaert[['Word','Conc.M']]
  # Remove any words that do not have a concreteness rating
  Brysbaert = Brysbaert[Brysbaert['Conc.M'].notna()]
  # Set the dataframe index to the word
  Brysbaert = Brysbaert.set_index('Word')
  # Take the Concreteness rating series out as a dictionary
  Brysbaert_dict = Brysbaert['Conc.M'].to_dict()
  
  return d_c_list, AoA_dict, Brysbaert_dict

In [ ]:
# Define a function to calculate a score based on values from an external resource
def extraneous_score_calculation(text, extraneous_dict):
  # Replace all non-alphanumeric characters with a space, then make the letters lowercase, and 
  # subsequently tokenize the words
  tokens = nltk.word_tokenize((re.sub("[^a-zA-Z0-9 ]", " ", text)).lower())
  # Create an empty array to add score values into
  score_array = []

  # For each token in the tokenize 'original_text'
  for tok in tokens:
    # Try to find the token in the extraneous dictionary and append its score to the array
    try:
      ind_score = extraneous_dict[tok]
      score_array.append(ind_score)
    # If unable to find the token, append a value of 0 to the array
    except:
      score_array.append(0)
  # Return a normalized score for the 'original_text' column by summing the scores together and dividing by 
  # the total number of tokens.
  return np.sum(score_array)/len(score_array)

In [ ]:
# Use extraneous_score_calculation function to calculate AoA and Brysbaert Concreteness Scores
def get_AoA_Brysbaert_features(new_text_data):
  # Calculate AoA Score using extraneous_score_calculation function
  new_text_data['AoA_score'] = new_text_data.original_text.apply(lambda x: extraneous_score_calculation(x, AoA_dict))

  # Calculate Brysbaert Score using extraneous_score_calculation function
  new_text_data['Brysbaert_score'] = new_text_data.original_text.apply(lambda x: extraneous_score_calculation(x, Brysbaert_dict))

  # Convert NaN in Both 'AoA_score' and 'Brysbaert_score' columns
  new_text_data['AoA_score'] = new_text_data['AoA_score'].fillna(0)
  new_text_data['Brysbaert_score'] = new_text_data['Brysbaert_score'].fillna(0)

  return new_text_data

In [ ]:
# Acquire additional features such as:
# 1) Normalized proportion of word tokens from Dale Chall list in 'original_text' column
# 2) Number of tokens in 'original_text' column
# 3) Average length of each word token in 'original_text' column
# 4) Largest length of a word token in 'original_text' column
# 5) Normalized proportion of non-alphanumeric characters in 'original_text' column
# 6) Normalized proportion of decimal digit characters in 'original_text' column
def get_more_features(list_of_docs, easy_word_list):
    # List for number of word tokens in text passage
    num_toks_l = []
    # List for number of dale_chall terms in text passage, normalized to length of text passage
    d_c_norm_l = []
    # Value of average word length for a textual passage
    avg_tok_len_l = []
    # Value of max word length for a textual passage
    max_tok_len_l = []
    # Number of Non-alphanumeric characters
    non_alpha_char_l =[]
    # Number of Characters total
    numbers_norm_l = [] #add ratio of number charicters to total
    # Generate a set of words based on the second input of the function (a list of words)
    s2=set(easy_word_list)
    
    
    for doc in list_of_docs:
        # Convert all letters to lowercase
        doc = doc.lower()
        #-------------------
        chars = re.findall('[^a-zA-Z0-9 ]', doc) # Find all non-alphanumeric characters (except whitespace)
        non_alpha = len(chars)/len(doc) # Calculate a Normalized Ratio of the number of non-alphanumeric characters to the length of the entire text passage
        non_alpha_char_l.append(non_alpha) # Append this ratio to the previously generated list
        #-------------------------------
        num_chars = re.findall('\d', doc) # Find all decimal digit characters
        numbers_norm = len(num_chars)/len(doc) # Calculate the normalized ratio to the length of the entire text passage
        numbers_norm_l.append(numbers_norm) # Append the calculated ratio to previously generated list
        #------------------------------
        toks = nltk.word_tokenize(doc) # Generate word tokens for each text passage using nltk.tokenize.word_tokenize
        num_toks = len(toks) # Count the number of tokens
        num_toks_l.append(num_toks) # Append the token count to previously generated list
        #------------------------------
        temp_list = [] # Create an empty temporary list
        # For each token created from word_tokenize
        for tok in toks:
            # Determine the length of the token, and append that length to the temporary list
            temp_list.append(len(tok))
            
        # Find the average token length
        avg_tok = sum(temp_list)/len(temp_list)
        # Append the average token length to previously generated list
        avg_tok_len_l.append(avg_tok)
        # Find the maximum token length
        max_t = max(temp_list)
        # Append the maximum token length to previously generated list
        max_tok_len_l.append(max_t)
        #------------------------------
        s1= set(toks) # Generate a set of tokens from previously made list of tokens
        num_d_c = len(s1.intersection(s2)) # Calculate the number of words that are also contained in the set of 'simple words' made previously
        d_c_norm = num_d_c/num_toks # Normalize the value to the total number of tokens
        d_c_norm_l.append(d_c_norm) # Append that normalized value to previously generated list
        #------------------------------
        
    # Generate an array of new features which can be added to the dataframe
    new_features = np.vstack(( np.asarray(d_c_norm_l), np.asarray(num_toks_l),  np.asarray(avg_tok_len_l), np.asarray(max_tok_len_l), 
                   np.asarray(numbers_norm_l), np.asarray(non_alpha_char_l) )).T

    return new_features

In [ ]:
# Function to count the number of a specific character within an 'original_text' column
# This function designed to help with identifying non-alphanumeric characters as special characters
# can have issue when using regex to search for them.
def count_num_of_specific_char(text, char_of_interest):
  # Reformat non-alphanumeric character as a set contained in brackets
  reformat_char = '[' + char_of_interest + ']'
  # Find all occurences of the character in a text, and count the total number of them
  num_specific_char = len(re.findall(reformat_char, text))
  # Return the total count of the non-alphanumeric characters
  return num_specific_char

In [ ]:
# Function to count the total number of non-whitespace characters
def count_num_of_non_ws(text):
  # Replace all whitespace characters with no text
  new_string = re.sub("\s", "", text)

  # Return the count the length of the new non-whitespace string
  return len(new_string)

In [ ]:
# Make Individual feature columns for the normalized proportion of the punctuation/non-alphanumeric characters
def get_punctuation_features(text_data):
  # Acquire all of the non-alphanumeric characters in a set
  all_punctuation = set(text_data.punctuation.sum())

  # Create a new empty list to track all columns added to dataframe
  new_columns = []

  # For each non-alphanumeric character
  for punc_mark in all_punctuation:
    # Create a new string for a potential column name
    new_col_name = 'norm_' + punc_mark

    # Try to count the number of entries of the specific character, and if so, add the name of the column to the list of new column names
    try:
      text_data[new_col_name] = text_data.punctuation.apply(lambda x: count_num_of_specific_char(x, punc_mark))
      text_data[new_col_name] = text_data[new_col_name] / text_data['num_non_ws_char']
      new_columns.append(new_col_name)
    # If there are issues, then continue to the next non-alphanumeric character
    except:
      continue
  # Return the dataframe and a list of these new columns
  return text_data, new_columns

In [ ]:
# Function to transform a text into an array of Parts-of-Speech (POS)
def POS_preprocessing(text):
  # Replace all non-alphanumeric or common punctuation with no text and output into a new string
  new_string = re.sub('[^a-zA-Z0-9 ,.!;:?()]', '', text)
  # Tokenize the new string
  word_tokens = nltk.word_tokenize(new_string)
  # Obtain the parts of speech tags for each of the words and put them into a list
  pos_tag_tokens = [pair[1] for pair in nltk.pos_tag(word_tokens)]
  # Concatenate each POS tag together into a single string
  pos_tag_tokens = " ".join(pos_tag_tokens)
  # Return the single string of POS tags
  return pos_tag_tokens

In [ ]:
# Lemmatize and tokenize text
def lemma_preprocessing(text):
  lemmatizer = nltk.WordNetLemmatizer()
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = text.lower()
  text = nltk.word_tokenize(text)
  text = [lemmatizer.lemmatize(word) for word in text]
  text = " ".join(text)
  return text

In [ ]:
new_text_data = preprocessing_training_data()
d_c_list, AoA_dict, Brysbaert_dict = load_external_resource_data()
new_text_data = get_AoA_Brysbaert_features(new_text_data)
new_features = get_more_features(new_text_data['original_text'], d_c_list)
new_text_data[['d_c_norm_1', 'num_toks_1', 'avg_tok_len_1', 'max_tok_len_1', 'num_char_norm_1', 'non_alphanumeric_1']] = new_features
new_text_data['num_non_ws_char'] = new_text_data['original_text'].apply(lambda x: count_num_of_non_ws(x))
new_text_data, punc_cols = get_punctuation_features(new_text_data)
new_text_data['pos_tag_tokens'] = new_text_data['original_text'].apply(lambda x: POS_preprocessing(x))
new_text_data['num_pos_tokens'] = new_text_data['pos_tag_tokens'].apply(lambda x: len(x.split()))
new_text_data['lemma_text'] = new_text_data['original_text'].apply(lambda x: lemma_preprocessing(x))
new_text_data = new_text_data.drop(columns=['punctuation'])
#new_text_data.head()

<ipython-input-12-cc2f911ccd95>:8: FutureWarning: Possible nested set at position 1
  num_specific_char = len(re.findall(reformat_char, text))
<ipython-input-14-60cef8cc6f5e>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  text_data[new_col_name] = text_data.punctuation.apply(lambda x: count_num_of_specific_char(x, punc_mark))
<ipython-input-17-15b3d35ea10b>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  new_text_data['pos_tag_tokens'] = new_text_data['original_text'].apply(lambda x: POS_preprocessing(x))
<ipython-input-17

In [ ]:
def determine_X_feat(df):
  all_features = df.columns.to_list()
  X_feat = []
  for feat in all_features:
    if feat != 'label':
      X_feat.append(feat)
  return X_feat

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

def scikit_column_transformer(text_df = new_text_data, text_type = 'original_text', vector_type = 'Count', scaler='Robust', ngrams_value=1, max_features_value=None, sequence_length=500, 
                              test_size=0.2, random_state=21):
  # Reduce the input dataframe to only include either the original_text or lemma_text columns
  if text_type == 'original_text':
    final_text_df = text_df.drop(columns=['lemma_text'])
    final_text_df = final_text_df.rename(columns={'original_text': 'text'})
  elif text_type == 'lemma_text':
    final_text_df = text_df.drop(columns=['original_text'])
    final_text_df = final_text_df.rename(columns={'lemma_text': 'text'})
  else:
    return 'Incorrect input for text_type argument'

  # Perform the Train-Test Split Based on Input Data
  X_feat = determine_X_feat(final_text_df)
  X_train, X_test, y_train, y_test = train_test_split(final_text_df[X_feat], final_text_df['label'], test_size=test_size, random_state=random_state)
  
  # Select Vectors for text data and POS data
  if vector_type == 'Count':
    text_vector = CountVectorizer(ngram_range=(1, ngrams_value),max_features=max_features_value)
    pos_vector = CountVectorizer(ngram_range=(1, ngrams_value), preprocessor=None, token_pattern=r'[^\s]+', lowercase=False)
  elif vector_type == 'Tfidf':
    text_vector = TfidfVectorizer(ngram_range=(1, ngrams_value), max_features=max_features_value)
    pos_vector = TfidfVectorizer(ngram_range=(1, ngrams_value), token_pattern=r'[^\s]+', lowercase=False)
  elif vector_type == 'Binary':
    text_vector = CountVectorizer(binary=True, ngram_range=(1, ngrams_value),max_features=max_features_value)
    pos_vector = CountVectorizer(binary=True, ngram_range=(1, ngrams_value), preprocessor=None, token_pattern=r'[^\s]+', lowercase=False)
  else:
    return 'Incorrect input for vector_type argument'

  # Select the desired scaler based on input string
  dict_of_scalers = {'Robust': RobustScaler(), 'MinMax': MinMaxScaler() , 'Standard': StandardScaler()}
  try:
    selected_feature_scaler = dict_of_scalers[scaler]
  except:
    return 'Incorrect input for scaler argument - must be either Count, MinMax or Standard'
  
  # Use Scikit-Learn Column Transformer to vectorize the text data and the POS data, and transform the additional features by selected scaler
  column_trans = ColumnTransformer([('vector_text', text_vector, 'text'), 
                                    ('vector_pos_tags', pos_vector, 'pos_tag_tokens')], 
                                   remainder = selected_feature_scaler)
  
  # Perform Fit_Transform on X_train and transform on X_test
  X_train_matrix = column_trans.fit_transform(X_train)
  X_test_matrix = column_trans.transform(X_test)

  return column_trans, X_train_matrix, y_train, X_test_matrix, y_test
